<a href="https://colab.research.google.com/github/sahiti3636/AIML_SESSION/blob/main/Module2_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval-Augmented Generation (RAG) — Simple Demo

In this notebook, we build a simple RAG system:
1. Store documents
2. Convert text into embeddings
3. Retrieve relevant context
4. Generate an answer using an LLM (Gemini)

## Install Dependencies



In [ ]:
!pip install langchain langchain-google-genai langchain_community langchain_core chromadb sentence-transformers pypdf

## Importing necessary libraries

In [1]:
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
import os
os.environ["GOOGLE_API_KEY"] = "PASTE_YOUR_API_KEY_HERE"

## Our Knowledge Base

Upload PDF

In [ ]:
from google.colab import files

uploaded = files.upload()

Load PDF as Documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

all_documents = []

for pdf_path in uploaded.keys():
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    all_documents.extend(docs)

Split into Chunks

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

documents = text_splitter.split_documents(all_documents)

## Create Embeddings + Store in Schema

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

In [7]:
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

## Retriever (Semantic Search)

In [8]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [15]:
query = "What is the main difference between active and passive electronic components?"

retrieved_docs = retriever.invoke(query)

In [ ]:
retrieved_docs

In [ ]:
len(retrieved_docs)

## Gemini LLM

In [18]:
llm = GoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

## Final RAG Answer

In [19]:
context = "\n".join(doc.page_content for doc in retrieved_docs)

prompt = f"""
You are an assistant that answers questions using the given context.

Rules:
- Use ONLY the information in the context.
- Do NOT use outside knowledge.
- If the context is insufficient, ask the user to rephrase the question
  or provide more information.

Answer Guidelines:
- Write a clear and complete explanation.
- Use 3–5 sentences.
- Prefer simple language suitable for a beginner.

Context:
{context}

Question:
{query}

Answer:
"""

In [ ]:
response = llm.invoke(prompt)
print(response)

In [ ]:
#Explain the 'Software as a Service' (SaaS) model in simple terms based on the document